In [3]:
import sys
sys.path.append("../src/") 
import os

import model_manipulation as mm
import cobra
import cplex 
import libsbml
import pandas as pd
import copy
from pathlib import Path
import memote
import csv
from cobra import Reaction
from datetime import datetime
from tabulate import tabulate

In [4]:
#Read 2-cell model
model = cobra.io.read_sbml_model("../model/ios2164_2cell.xml")
#Estimate inf
inf = 1e6
model.solver = 'gurobi'

now = datetime.now()

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-12


#Add transgenic reactions to model localized to specific compartment

Transgenic (Line 29 - Ermakova et al)
PEPC = Chloroplastic in M & V (rxn id: PPCc)
PPDK = Chloroplastic in M and BS (rxn id: PPDKs)
NADP-ME = Mitochondrial in M (rxn id: MDHys)
MDH = Chloroplastic in M and BS; Mitochondrial in M  (rxn id: MDH2s, MDH2c)
CA = Cytosolic in M (rxn id: hco3ec)

In WT:
PEPC = Cytosolic in V
PPDK = No signal
NADP-ME = No signal
MDH = Mitochondrial in M 
CA = No signal

Difference:
PEPC = Only in M
PPDK = Chloroplastic M & BS
NADP-ME = Only in M
MDH = Chloroplastic in M/BS
CA = Cytosolic in M



In [5]:
#This codeblock is to define some functions that will be used for modelling.


#Define linear relationship between PPFD and Cellular maintainance costs
#This formula comes from Topfer et al (2020) where she defined NGAM in a linear relationship with incident light
def compute_ngam_atp(ppfd):
    v_atp = 0.0049*ppfd + 2.7851
    return v_atp


#This function is used to set the inputs to the model used. 
def define_model_medium(model, co2, o2, ppfd, 
                        medium_dir='../misc/photo_medium.csv', no3=inf, h2o=inf, h=inf, 
                        nh4=inf, pi=inf, inf=1e6):
    model_photo_media = mm.read_medium_csv(medium_dir, model)
    model_photo_media['EX_no3(e)'] = no3
    model_photo_media['EX_h2o(e)'] = h2o
    model_photo_media['EX_h(e)'] = h
    model_photo_media['EX_nh4(e)'] = nh4
    model_photo_media['EX_co2(e)'] = co2
    model_photo_media['EX_o2(e)'] = o2
    model_photo_media['EX_photonVis(e)'] = ppfd
    model_photo_media['EX_pi(e)'] = pi
    #Set set model medium as model
    print('Added model medium')
    return model_photo_media

    
def turn_off_cofac_cycles(model, inact_dir='../misc/leaf_inactivated.tsv'):
    file = csv.reader(open(inact_dir), delimiter='\t')
    leaf_inactive_rxns = list()
    for rows in file:
        row_m = str()
        row_bs = str()
        for rxns in rows:
            row_m += str(rxns) + "_M"
            row_bs += str(rxns) + "_BS"
        leaf_inactive_rxns.append(row_m)
        leaf_inactive_rxns.append(row_bs)
        
    for rxns in model.reactions:
        if rxns.id in leaf_inactive_rxns:
            rxns.bounds = (0,0)
    print('Successfully turned off cofactor-cycling reactions')

    
# #Add constraints to model
#This code block contains constraints that would simulate the assimilation rates of bs and m cells in a two-cell system (such as those seen near the midvein region of rice leaves)
# #BS photon flux must be the same/less than M flux (Adapted from B&B, 2019)
# photon_import = model.reactions.get_by_id("EX_photonVis(e)")
def add_tissue_constraints(model):
    BS_photon_import = model.reactions.PRISM_white_LED_BS
    M_photon_import = model.reactions.PRISM_white_LED_M

    #Set photon flux ratio to 10:1
    photon_flux = mm.set_fix_flux_ratio({M_photon_import.id:10, BS_photon_import.id:1},model)
    model.add_cons_vars(photon_flux)

    co2tex_m = model.reactions.CO2tex_M
    co2tex_bs = model.reactions.CO2tex_BS

    #CO2 Uptake must be constrained to A (Net assimilation rate) which is around 29 +- 1.2 umol CO2 m-2 s-1 for wild type rice
    #CO2 BS intake must be limited owing to its position in the leaf tissue

    # # #CO2 intake needs to be mostly in the M cells. Not sure of the exact value:
    # # Values from Von Caemmerer (2000) in intermediate C3-C4 photosynthetic plants is 1 mmol CO2 m-2 s-1 per 20 CO2 m-2 s-1, which puts it at 20:1 ratio
    # # #try checking Danila et al. (2016) on ratio of surface area of M cell to BS cell
    # # #I'll assume that the ratio is at 20:1 in the meantime, meaning that there is minimal gas exchange into the BS cell that would equate to around ~1 umol CO2 m-2s-1
    co2_ratio_cons = mm.set_fix_flux_ratio({co2tex_m.id:10, co2tex_bs.id:1},model)
    co2_ratio_cons.name = 'co2_ratio_cons'
    model.add_cons_vars(co2_ratio_cons)

    #o2 ratio constraint
    o2tex_bs = model.reactions.O2tex_BS
    o2tex_m = model.reactions.O2tex_M

    o2_ratio_cons = mm.set_fix_flux_ratio({o2tex_m.id:10, o2tex_bs.id:1},model)
    o2_ratio_cons.name = 'o2_ratio_cons'
    model.add_cons_vars(o2_ratio_cons)

    #no tissue-specific constraints will be set for Nitrates, h2o and other inorganic cofactors as 
    #they are transported via the apoplastic pathway rather than the symplast

# #This code block contains constraints specific for enzyme rate constraints
#This approach is derived from Bogart & Myers (2016) where they constrained the enzyme rate 
#fluxes in each of the 2-cell segments to a specific upper bound while keeping the lower bound
#At 0. For reversible reactions the lower bounds are set to the same value

def add_enzyme_constraints(model):
    #Maximum values for constraints
    wt_pepc = 0 #umol m-2 s-1 #Note: Need to constrain it to 0 pala since no PEPC was detected in either cell type
    wt_mdh = 11.18 #umol m-2 s-1
    wt_nadp_me = 0.14 #umol m-2 s-1
    wt_ppdk = 0.31 #umol m-2 s-1
    wt_CA = 7.5 #umol m-2  s-1 bar-1 (Constrained to CO2 amounting to 400-500 mbar) (1 bar = 15.74 umol m-2 s-1 CA activity)

    #PEPC constraint (Reaction id: PPCc)
    #Need to constrain it to 0 since reaction is only detected in Vascular tissue
    pepc_BS = model.reactions.PPCc_BS
    pepc_M = model.reactions.PPCc_M

    wt_pepc_cons = model.problem.Constraint(pepc_BS.flux_expression 
                                            + pepc_M.flux_expression, 
                                            lb = 0, ub = wt_pepc)
    wt_pepc_cons.name = 'wt_pepc_cons'
    model.add_cons_vars(wt_pepc_cons)

    #PPDK constraints (Reaction id: PPDKs) (note that this is found in the chloroplast?) 
    #Not detected via immunolocalization but enzyme activity is detected

    ppdks_BS = model.reactions.PPDKs_BS
    ppdks_M = model.reactions.PPDKs_M
    wt_ppdks_cons = model.problem.Constraint(ppdks_BS.flux_expression 
                                             + ppdks_M.flux_expression, 
                                             lb = 0, ub = wt_ppdk)
    wt_ppdks_cons.name = 'wt_ppdks_cons'
    model.add_cons_vars(wt_ppdks_cons)
    #Malate Dehydrogenase 
    #Only mitochondrial in WT Rice M cells
    mdhm_M = model.reactions.MDHm_M


    wt_mdh_cons = model.problem.Constraint(mdhm_M.flux_expression,
                                           lb= 0, ub=wt_mdh)
    wt_mdh_cons.name = "wt_mdh_cons"
    model.add_cons_vars(wt_mdh_cons)

    #NADP-ME (Since no signal is detected in WT, no locational constraints are imposed)
    #Let's see if I can force it to have a small amount of flux 
    nadp_me_M = model.reactions.MDHys_M
    nadp_me_BS = model.reactions.MDHys_BS

    wt_nadpme_cons = model.problem.Constraint(nadp_me_M.flux_expression
                                             + nadp_me_BS.flux_expression,
                                             lb= 0, ub=wt_nadp_me)
    wt_nadpme_cons.name = "wt_nadpme_cons"
    model.add_cons_vars(wt_nadpme_cons)
    #Weird, it causes the activity of RBCL to significantly vary from the actual values


    #I should add constraints for Carbonic Anhydrase
    #I should constrain it to 0.4 ubar, which would constitute ambient CO2 partial pressure
    #Flux is reversible so constraints are bi-directional


    hco3es_m = model.reactions.HCO3Es_M.flux_expression
    hco3ec_m = model.reactions.HCO3Ec_M.flux_expression
    hco3em_m = model.reactions.HCO3Em_M.flux_expression
    hco3es_bs = model.reactions.HCO3Es_BS.flux_expression
    hco3ec_bs = model.reactions.HCO3Ec_BS.flux_expression
    hco3em_bs = model.reactions.HCO3Em_BS.flux_expression

    ca_cons = model.problem.Constraint(hco3es_m + hco3ec_m + hco3em_m 
                                       + hco3es_bs + hco3ec_bs + hco3em_bs,
                                      lb = -wt_CA, ub = wt_CA)
    ca_cons.name = 'Carbonic_anhydrase_constraint'
    model.add_cons_vars(ca_cons)
    #Rbcl constaints
    #Retrieve flux expressions oof each RBCl reaction
    rbpc_M = model.reactions.RBPCs_M.flux_expression
    rbpc_BS = model.reactions.RBPCs_BS.flux_expression
    rbpo_M = model.reactions.RBPOs_M.flux_expression
    rbpo_BS = model.reactions.RBPOs_BS.flux_expression

    #Constraint such that it is limited to 132 umol m-2 s-1
    rbcl_vcmax_cons = model.problem.Constraint(rbpc_M + rbpc_BS, lb = 0, ub= 132)
    rbcl_vcmax_cons.name='rbcl_vcmax_cons'
    model.add_cons_vars(rbcl_vcmax_cons)
    #Constraints for rbcl flux such that v_c/v_o = 3 or higher.
    rbcl_vcvo = model.problem.Constraint(3*(rbpo_M + rbpo_BS) 
                                         - 1*(rbpc_M + rbpc_BS),
                                         lb=0,ub=1000)
    rbcl_vcvo.name = 'rbcl_vc/vo_ratio'
    model.add_cons_vars(rbcl_vcvo)
    
    
    #Turn off the RBPC2s reactions since we already defined the constraints above
    model.reactions.RBPC2s_M.bounds = (0,0)
    model.reactions.RBPC2s_BS.bounds = (0,0)
    
    
    
    #What if I simply constrained that of the M cell one to 3:1?
    #This constraint is pretty good actually. 
    #This allows the system to be set at a specific Vc/Vo rate while still allowing local variation 
    #wherein Rubisco may act in an uncoupled fashion and may have favorable internal vc/vo rates.
# #This code block is to set a constraint such that M-to-BS cell NGAM ratio is 10-to-1 
# #Similar to what Moreno-Villena (2021) (preprint) had done 

#This function takes two arguments: the model and the maximal  ppfd input to the system
def add_ngam_cons(model, ppfd): 
    ngam_atp_m = mm.get_rxn(model, 'ngam_atp_c_M')
    ngam_atp_bs = mm.get_rxn(model, 'ngam_atp_c_BS')
    ngam_atp_m.bounds = (0,1000)
    ngam_atp_bs.bounds = (0,1000)
    ngam_ratio = mm.set_fix_flux_ratio({ngam_atp_m.id:10, ngam_atp_bs.id:1}, model)
    ngam_ratio.name = 'ngam_BS/M_ratio'
    model.add_cons_vars(ngam_ratio)

    #Retrieve NGAM reactions
    ngam_nadphox_c_M = mm.get_rxn(model, 'ngam_nadphox_c_M')
    ngam_nadphox_s_M = mm.get_rxn(model, 'ngam_nadphox_s_M')
    ngam_nadphox_m_M = mm.get_rxn(model, 'ngam_nadphox_m_M')
    ngam_nadphox_c_BS = mm.get_rxn(model, 'ngam_nadphox_c_BS')
    ngam_nadphox_s_BS = mm.get_rxn(model, 'ngam_nadphox_s_BS')
    ngam_nadphox_m_BS = mm.get_rxn(model, 'ngam_nadphox_m_BS')


    #Set Fixed fluxes
    nadphox_c_s_M = mm.set_fix_flux_ratio({ngam_nadphox_c_M.id:1, ngam_nadphox_s_M.id:1},model)
    nadphox_c_s_M.name = "nadphox_cs_ratio_M"
    nadphox_s_m_M = mm.set_fix_flux_ratio({ngam_nadphox_s_M.id:1, ngam_nadphox_m_M.id:1}, model)
    nadphox_s_m_M.name = "nadphox_sm_ratio_M"

    nadphox_c_s_BS = mm.set_fix_flux_ratio({ngam_nadphox_c_BS.id:1, ngam_nadphox_s_BS.id:1},model)
    nadphox_c_s_BS.name = "nadphox_cs_ratio_BS"
    nadphox_s_m_BS = mm.set_fix_flux_ratio({ngam_nadphox_s_BS.id:1, ngam_nadphox_m_BS.id:1}, model)
    nadphox_s_m_BS.name = "nadphox_sm_ratio_BS"

    #Add constraints
    model.add_cons_vars(nadphox_c_s_M)
    model.add_cons_vars(nadphox_s_m_M)
    model.add_cons_vars(nadphox_c_s_BS)
    model.add_cons_vars(nadphox_s_m_BS)

    #Retrieve flux expressionns
    fex_nadphox_c_M =  mm.get_flux_exp(model, ngam_nadphox_c_M)
    fex_nadphox_s_M = mm.get_flux_exp(model, ngam_nadphox_s_M)
    fex_nadphox_m_M = mm.get_flux_exp(model, ngam_nadphox_m_M)

    fex_nadphox_c_BS =  mm.get_flux_exp(model, ngam_nadphox_c_BS)
    fex_nadphox_s_BS =  mm.get_flux_exp(model, ngam_nadphox_s_BS)
    fex_nadphox_m_BS =  mm.get_flux_exp(model, ngam_nadphox_m_BS)

    fex_atp_c_M = mm.get_flux_exp(model, ngam_atp_m)
    fex_atp_c_BS =  mm.get_flux_exp(model, ngam_atp_bs)

    #Set the constraint between ATP:NADPH NGAM to 3:1
    nadphox_atpase = model.problem.Constraint(3*(fex_nadphox_c_M + fex_nadphox_s_M + fex_nadphox_m_M
                                                       + fex_nadphox_c_BS + fex_nadphox_s_BS + fex_nadphox_m_BS) 
                                         - 1*(fex_atp_c_M + fex_atp_c_BS),
                                         lb=0,ub=0)
    nadphox_atpase.name = "nadphox_atpase_ratio"
    model.add_cons_vars(nadphox_atpase)
    
    
    
    ngam_value = compute_ngam_atp(ppfd)
    ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                        fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
    ngam_cons.name = 'NGAM_constraint'
    model.add_cons_vars(ngam_cons)
    
#This code  block gives a snapshot of the relevant fluxes on each of the cell types based on the saved sample_fluxes values above

def print_summary(sample_fluxes_df):
    print('rbcl M cell: ', sample_fluxes['RBPCs_M'], 'rbcl BS cell: ',sample_fluxes['RBPCs_BS'])
    print('rbcl M cell (photorespiration)', sample_fluxes['RBPOs_M'], 'rbcl BS cell (PR)', sample_fluxes['RBPOs_BS'])
    print('vc/vo M:', sample_fluxes['RBPCs_M']/sample_fluxes['RBPOs_M'], 'vc/vo BS:', sample_fluxes['RBPCs_BS']/sample_fluxes['RBPOs_BS'])
    print('RBPC2s_M', sample_fluxes['RBPC2s_M'], 'RBPC2s_BS', sample_fluxes['RBPC2s_BS'])
    print('PEPC M', sample_fluxes['PPCc_M'], 'PEPC BS', sample_fluxes['PPCc_BS'])
    print('Carbonic Anhydrase (Cytosolic) M', sample_fluxes['HCO3Ec_M'], 'Carbonic Anhydrase (Cytosolic) BS', sample_fluxes['HCO3Ec_BS'])
    print('NADP-ME M', sample_fluxes['MDHys_M'], 'NADP-ME BS', sample_fluxes['MDHys_BS'])
    print('Biomass M: ', sample_fluxes['Straw_Biomass_M'], 'Biomass BS', sample_fluxes['Straw_Biomass_BS'])
    print('Phloem M: ', sample_fluxes['DM_Phloem_M'], 'Phloem BS', sample_fluxes['DM_Phloem_BS'])
    print('co2 consumption M', sample_fluxes['CO2tex_M'], 'co2 consumption BS', sample_fluxes['CO2tex_BS'])
    print('o2 consumption M', sample_fluxes['O2tex_M'], 'o2 consumption BS', sample_fluxes['O2tex_BS'])
    print('Photosystem II M', sample_fluxes['PSIINC_M'], 'PSII BS', sample_fluxes['PSIINC_BS'])
    print('PSI M', sample_fluxes['PSIMR_M'], 'PSI BS', sample_fluxes['PSIMR_BS'])
    print('PPFD M: ', sample_fluxes['PRISM_white_LED_M'], 'PPFD BS: ', sample_fluxes['PRISM_white_LED_BS'])
    print('ATP synthesis (stromal) M', sample_fluxes['ATPSs_M'], 'ATP synthase (mit) M', sample_fluxes['ATPSm_M'])
    pd_rxn = [x for x in model.reactions if "pd" in x.id and "h2o" not in x.id]
    pd_abs_flux = 0
    for pds in pd_rxn:
        pd_abs_flux += abs(sample_fluxes[pds.id])
    
    print('pd_abs_flux: ', pd_abs_flux)

In [6]:
#initialize list of transgenic reactions to add  to model

def add_trans_reactions(model):
    trans_list = list()


    #Transgenic PEPC copy
    #PEPC = Chloroplastic in M & V (rxn id: PPCc)
    trans_ppcs = Reaction('trans_PPCs_M')
    trans_ppcs.name = "Phosphoenolpyruvate carboxylase, plastidic (Transgenic)"

    co2_s0 = model.metabolites.co2_s0
    h_s0 = model.metabolites.h_s0
    pep_s0 = model.metabolites.pep_s0
    h2o_s0 = model.metabolites.h2o_s0
    oaa_s0 = model.metabolites.oaa_s0
    pi_s0 = model.metabolites.pi_s0


    #Add metabolites, bounds, and subsystem
    trans_ppcs.add_metabolites({co2_s0:-1, h_s0:-1, pep_s0:-1, h2o_s0:1, oaa_s0:1, pi_s0:1})
    trans_ppcs.bounds= model.reactions.PPCc_M.bounds
    trans_ppcs.subsystem = model.reactions.PPCc_M.subsystem


    trans_list.append(trans_ppcs)


    #Transgenic PPDK Copy
    #Since it already exists I'll just copy and readd it
    trans_ppdks_m = Reaction('trans_PPDKs_M')
    trans_ppdks_m.add_metabolites(model.reactions.PPDKs_M.metabolites)
    trans_ppdks_m.bounds = model.reactions.PPDKs_M.bounds
    trans_ppdks_m.name = "Pyruvate phosphate dikinase, plastidic (Transgenic)"

    trans_ppdks_bs = Reaction('trans_PPDKs_BS')
    trans_ppdks_bs.add_metabolites(model.reactions.PPDKs_BS.metabolites)
    trans_ppdks_bs.bounds = model.reactions.PPDKs_BS.bounds
    trans_ppdks_bs.name = "Pyruvate phosphate dikinase, plastidic (Transgenic)"

    trans_list.append(trans_ppdks_m)
    trans_list.append(trans_ppdks_bs)


    #Transgenic NADP-ME
    #NADP-ME = Mitochondrial in M
    trans_nadp_me = Reaction('trans_MDHym_M')

    #retrieve reactants
    mal_m0 = model.metabolites.get_by_id('mal-L_m0')
    nadp_m0 = model.metabolites.nadp_m0
    h_m0 = model.metabolites.h_m0
    nadph_m0 = model.metabolites.nadph_m0
    oaa_m0 = model.metabolites.oaa_m0

    #Add to rxn
    trans_nadp_me.add_metabolites({mal_m0:-1, nadp_m0:-1, h_m0:1, nadph_m0:1, oaa_m0:1})
    #Add bounds
    trans_nadp_me.bounds=(-inf, inf)

    trans_list.append(trans_nadp_me)


    #Malate Dehydrogenase, mitochondrial (M cell)
    trans_MDHm_M = Reaction('trans_MDHm_M')
    trans_MDHm_M.name = 'Malate Dehydrogenase, Mitochondrial'
    trans_MDHm_M.add_metabolites(model.reactions.MDHm_M.metabolites)
    trans_MDHm_M.subsystem = model.reactions.MDHm_M.subsystem

    trans_list.append(trans_MDHm_M)

    #Malate dehydrogenase, plastidic (M cell)
    trans_MDHs_M = Reaction('trans_MDHs_M')
    trans_MDHs_M.name = 'Malate Dehydrogenase, Plastidic'
    trans_MDHs_M.add_metabolites(model.reactions.MDHs_M.metabolites)
    trans_MDHs_M.subsystem = model.reactions.MDHs_M.subsystem

    trans_list.append(trans_MDHs_M)

    #Malate dehydrogenase, plastidic(BS Cell)
    trans_MDHs_BS = Reaction('trans_MDHs_BS')
    trans_MDHs_BS.name = 'Malate Dehydrogenase, Plastidic'
    trans_MDHs_BS.add_metabolites(model.reactions.MDHs_BS.metabolites)
    trans_MDHs_BS.subsystem = model.reactions.MDHs_BS.subsystem

    trans_list.append(trans_MDHs_BS)


    #Trans CA
    #Cytosolic in M
    trans_hco3ec_M = Reaction('trans_hco3ec_M')
    trans_hco3ec_M.name = 'carbonic anhydrase, cytosolic'
    trans_hco3ec_M.add_metabolites(model.reactions.HCO3Ec_M.metabolites)
    trans_hco3ec_M.bounds = model.reactions.HCO3Ec_M.bounds

    trans_hco3ec_M.subsystem = model.reactions.HCO3Ec_M.subsystem
    trans_list.append(trans_hco3ec_M)


    #Bulk add to model
    model.add_reactions(trans_list)

#     print("Successfully added transgenic reactions")

In [7]:
def add_trans_constraints(model):
    #The following code block is done to
    #Define rates
    trans_pepc_rates = 7.01 #umol m-2 s-1
    trans_ppdks_rates = 3.66 #umol m-2 s-1
    trans_mdh_rates = 152.87 #umol m-2 s-1
    trans_nadp_me_rates = 0.60 #umol m-2 s-1
    trans_CA_rates = 8 #umol m-2  s-1 bar-1 (Constrained to CO2 amounting to 400-500 mbar) (1 bar = 15.74 umol m-2 s-1 CA activity)


    #Generate constraints on top of WT constraints

    #Retrieve trans reactions
    trans_PPCs_M = mm.get_rxn(model, 'trans_PPCs_M')
    trans_ppdks_M  = mm.get_rxn(model, 'trans_PPDKs_M')
    trans_ppdks_BS = mm.get_rxn(model, 'trans_PPDKs_BS')
    trans_MDHm_M = mm.get_rxn(model, 'trans_MDHm_M')
    trans_MDHs_M = mm.get_rxn(model, 'trans_MDHs_M')
    trans_MDHs_BS = mm.get_rxn(model, 'trans_MDHs_BS')
    trans_MDHym_M = mm.get_rxn(model, 'trans_MDHym_M')
    trans_hco3ec_M = mm.get_rxn(model, 'trans_hco3ec_M')


    #PEPC constraint
    trans_pepc_cons = model.problem.Constraint(trans_PPCs_M.flux_expression
                                            +pepc_BS.flux_expression 
                                            + pepc_M.flux_expression, 
                                            lb = 0, ub = trans_pepc_rates)

    model.add_cons_vars(trans_pepc_cons)


    trans_ppdks_cons = model.problem.Constraint( 
        trans_ppdks_BS.flux_expression +trans_ppdks_M.flux_expression 
        +ppdks_BS.flux_expression + ppdks_M.flux_expression, 
                                             lb = 0, ub = trans_ppdks_rates)
    trans_ppdks_cons.name = 'trans_ppdks_cons'
    model.add_cons_vars(trans_ppdks_cons)



    trans_mdh_cons =  model.problem.Constraint(
       trans_MDHm_M.flux_expression + mdhm_M.flux_expression
        + trans_MDHs_M.flux_expression + trans_MDHs_BS.flux_expression,
                                           lb= 0, ub=trans_mdh_rates)

    trans_mdh_cons.name = "trans_mdh_cons"
    model.add_cons_vars(trans_mdh_cons)


    trans_nadpme_cons = model.problem.Constraint(
        trans_MDHym_M.flux_expression + nadp_me_M.flux_expression + nadp_me_BS.flux_expression,
                                             lb= 0, ub=trans_nadp_me_rates)
    trans_nadpme_cons.name = "trans_nadpme"
    model.add_cons_vars(trans_nadpme_cons)


    trans_ca_cons = model.problem.Constraint(trans_hco3ec_M.flux_expression + hco3es_m + hco3ec_m + hco3em_m 
                                       + hco3es_bs + hco3ec_bs + hco3em_bs,
                                      lb = -trans_CA_rates, ub = trans_CA_rates)
    ca_cons.name = 'Trans_CA_cons'
    model.add_cons_vars(trans_ca_cons)

#     print('Successfully added Transgenic-specific constraints')

This code block is to add the constraints for ngam nadphox, which for some reason doesn't save when it was generated using the curation script. I think this is because the constraint objects are only retrieved from the bounds encoded in the SBML.


In [13]:
# # This code block is to test whether a single iteration works. Maybe I should implement the kinetic constraints by Von Caemmerer to obtain proper 
# #proper values? I think that would be a better approach, I think.


with model:
    for rxns in model.reactions:
        if rxns.id in leaf_inactive_rxns: #Turn off all reactions in the list loaded above
             rxns.bounds = (0,0)
                
    model_media['EX_photonVis(e)'] = 1000
    model_media['EX_co2(e)'] = 100
    model_media['EX_o2(e)'] = 100#2.2618
    model.medium = model_media
    
#     #Set NGAM
    ngam_value = compute_ngam_atp(200)
    ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                        fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
    model.add_cons_vars(ngam_cons)
    
    
#Constrain O2 intake to 3.312 mmol O2 gcw -1 d-1 ~~ 2.2618 umol O2 m-2 s-1 (from Lakshmanan et al. 2016)
    #Change objective function to mature leaf
    model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
    model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
    mm.get_rxn(model,'DM_Phloem_BS').objective_coefficient = 1
    sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
    
    sample_fluxes_df = sample_fluxes.to_frame()
    #Remove thermodynamically infeasible loops
#     sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes)
    
    
    
file_name = 'Testsoln-WT-{}.csv'.format(now.strftime("%Y%m%d-%H:%M"))
soln_path = '../flux_results/test_solns/{}'.format(file_name)
out = open(soln_path, 'w')
sample_fluxes.to_csv(out, sep='\t')
out.close()

print('Solution saved at ', soln_path)
    

Solution saved at  ../flux_results/test_solns/Testsoln-WT-20220503-15:55.csv


In [ ]:
datetime.time(datetime.now())

In [ ]:
# This code block is to test which of the reactions in the leaf inactivated list produces infeasible loops
phloem_producing_rxn = list()

for turn_on_rxn, rxn_bounds in leaf_intact_bounds_dict.items():
    print('Producing solution for: ', turn_on_rxn, '...')
    with model:
        #Turn off all reactions first
        for rxns in model.reactions:
            if rxns.id in leaf_inactive_rxns: #Turn off all reactions in the list loaded above
                 rxns.bounds = (0,0)

        model_media['EX_photonVis(e)'] = 1000
        model_media['EX_co2(e)'] = 100
        model_media['EX_o2(e)'] = 100#2.2618
        model.medium = model_media

        #Set NGAM
        ngam_value = compute_ngam_atp(200)
        ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                            fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
        model.add_cons_vars(ngam_cons)
        
        #Change objective function to mature leaf
        model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
        model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
        mm.get_rxn(model,'DM_Phloem_BS').objective_coefficient = 1
        
        #Open up reaction bounds in the list
        model.reactions.get_by_id(turn_on_rxn).bounds = rxn_bounds
        
        #Get solution
        sample_fluxes = cobra.flux_analysis.pfba(model).fluxes.to_frame()
        
        if sample_fluxes['fluxes']['DM_Phloem_BS'] != 0:
            phloem_producing_rxn.append(turn_on_rxn)
         
print('Here are a list of reactions that allow Phloem to be produced without co-factor cycling: ')
for items in phloem_producing_rxn:
    print(items)

In [ ]:
#Let's try opening up FDNHc only


with model:
    #Turn off all reactions first
    for rxns in model.reactions:
        if rxns.id in leaf_inactive_rxns: #Turn off all reactions in the list loaded above
             rxns.bounds = (0,0)

    model_media['EX_photonVis(e)'] = 1000
    model_media['EX_co2(e)'] = 100
    model_media['EX_o2(e)'] = 100#2.2618
    model.medium = model_media

    #Set NGAM
    ngam_value = compute_ngam_atp(200)
    ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                        fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
    model.add_cons_vars(ngam_cons)

    #Change objective function to mature leaf
    model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
    model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
    mm.get_rxn(model,'DM_Phloem_BS').objective_coefficient = 1

    #Open up reaction bounds in the list
    model.reactions.FDHNc_M.bounds = leaf_intact_bounds_dict['FDHNc_M']
    model.reactions.FDHNc_BS.bounds = leaf_intact_bounds_dict['FDHNc_BS']
    #Get solution
    sample_fluxes = cobra.flux_analysis.pfba(model).fluxes.to_frame()
    
#Save to csv
    
    
soln_path = '../flux_results/test_solns/Test-ccf-vs-no-ccf-1cell.csv'
out = open(soln_path, 'w')
sample_fluxes.to_csv(out, sep='\t')
out.close()

print('Solution saved at ', soln_path)

In [ ]:
#Let's try closing all cofactor-recycling reactions except FDHNc and let's check their solutions.

In [ ]:
#Let's try turning on FDHNc and see how the model reacts.

with model:
    #Turn off all reactions first
    for rxns in model.reactions:
        if rxns.id in leaf_inactive_rxns: #Turn off all reactions in the list loaded above
             rxns.bounds = (0,0)

    model_media['EX_photonVis(e)'] = 1000
    model_media['EX_co2(e)'] = 100
    model_media['EX_o2(e)'] = 100#2.2618
    model.medium = model_media

    #Set NGAM
    ngam_value = compute_ngam_atp(200)
    ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                        fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
    model.add_cons_vars(ngam_cons)

    #Change objective function to mature leaf
    model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
    model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
    mm.get_rxn(model,'DM_Phloem_BS').objective_coefficient = 1

    #Open up reaction bounds in the list
    model.reactions.FDHNc_M.bounds = leaf_intact_bounds_dict['FDHNc_M']
    model.reactions.FDHNc_BS.bounds = leaf_intact_bounds_dict['FDHNc_BS']
    #Get solution
    sample_fluxes = cobra.flux_analysis.pfba(model).fluxes.to_frame()
    
#Save to csv
    
    
soln_path = '../flux_results/test_solns/Test-soln-no-cofac-turned-on-FNDHc_M.csv'
out = open(soln_path, 'w')
sample_fluxes.to_csv(out, sep='\t')
out.close()

print('Solution saved at ', soln_path)

In [ ]:
#May 3, 2022

#Let's compare solutions between yung non-CCF, non-pFBA, and yung FBA lang na solutions
# This code block is to test whether a single iteration works. Maybe I should implement the kinetic constraints by Von Caemmerer to obtain proper 
#proper values? I think that would be a better approach, I think.
flux_loopless_comparison_df = pd.DataFrame(index=list(i.id for i in model.reactions))


#
with model:
    
    model_media['EX_photonVis(e)'] = 500
    model_media['EX_co2(e)'] = 29
    model_media['EX_o2(e)'] = 2.26
    model.medium = model_media
    add_tissue_constraints(model)
    add_enzyme_constraints(model)
    add_ngam_cons(model, 500)

    print('ppfd: 500')
    print('o2 intake:', model_media['EX_o2(e)'])
    print('Objective function: DM_Phloem')
    #Constrain O2 intake to 3.312 mmol O2 gcw -1 d-1 ~~ 2.2618 umol O2 m-2 s-1 (from Lakshmanan et al. 2016)
    #Change objective function to mature leaf
    model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 1
    model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 1
    mm.get_rxn(model,'DM_Phloem_BS').objective_coefficient = 0
    
    #FBA only
    print('solving fba only')
    fluxes_fba = model.optimize().to_frame()
    flux_loopless_comparison_df['FBA only'] = fluxes_fba['fluxes']
    
    #pFBA only
    print('solving pfba only')
    fluxes_pfba = cobra.flux_analysis.pfba(model).fluxes.to_frame()
    flux_loopless_comparison_df['pFBA only'] = fluxes_pfba['fluxes']
    
                                                                               
# #     #Generate solution with CCF   
    
#     #for fba/ccf
    print('solving fba/ccf')
    fluxes_fba = model.optimize()
    fluxes_fba_ccf = cobra.flux_analysis.loopless_solution(model, fluxes_fba).to_frame()
    flux_loopless_comparison_df['FBA/ccf'] = fluxes_fba_ccf['fluxes']
        
    
    #for pfba/ccf
    print('solving pfba/ccf')
    fluxes_pfba = cobra.flux_analysis.pfba(model)
    fluxes_pfba_ccf = cobra.flux_analysis.loopless_solution(model, fluxes_pfba).to_frame()
    flux_loopless_comparison_df['pFBA/ccf'] = fluxes_pfba_ccf['fluxes']
    
    #Let's turn off yung mga looping reaction

    for rxns in model.reactions:
        if rxns.id in leaf_inactive_rxns:
            model.reactions.get_by_id(rxns.id).bounds = (0,0)
    print('Turned off Leaf co-factor recycling reactions')
    #Generate pfba solution
    print('solving nocofac/pfba')
    fluxes_pfba = cobra.flux_analysis.pfba(model).fluxes.to_frame()
    flux_loopless_comparison_df['no-cofactor-recyc/pfba'] = fluxes_pfba_ccf['fluxes']
    
    #pfba/ccf/no-cofac
    print('solving fluxes_pfba_nocofac')
    fluxes_pfba_nocofac = cobra.flux_analysis.pfba(model).fluxes
    fluxes_pfba_ccf_nocofac = cobra.flux_analysis.loopless_solution(model, fluxes_pfba_nocofac).to_frame()
    flux_loopless_comparison_df['no-cofactor-recyc/pfba/ccf'] = fluxes_pfba_ccf_nocofac['fluxes']
    

    


print(tabulate(flux_loopless_comparison_df, headers='keys', tablefmt='psql'))

    
file_path = '../flux_results/test_solns/Benchmarking-infeasibility-loops-Biomass-objective.csv'
out = open(file_path, 'w')
flux_loopless_comparison_df.to_csv(out, sep='\t')
out.close()

print('saved output to ', file_path)

In [ ]:
#Constraining O2 consumption to around 3.312 forces water to be consumed by the system, unlike when O2 is unconstrained.
#However, the same behavior when it comes to CO2 assimilation still is being produced.
#Constraining O2 consumption also forces Nitrate consumption to be constrained to realistic levels (when compared to non-constrained O2 where consumption was 10-fold )
#I suspect that this is definitely due to how PPFD flux is structured. 
#A cop-out to this would be to double PPFD per run and measuring PPFD flux in the specific reactions instead.

In [ ]:
#I can probably do the benchmarking right now. The model behaves quite interestingly actually

In [ ]:
#Note: Turning off other prism reactions (red, green, blue, etc) while retaining only white light reactions lowers net carbon assimilation
#I think this is in consideration on how white light is partitioned to several wavelengths, which prevents it from being fully utilized.

In [ ]:
#Other artifacts as of the moment include the following:
#model doesn't produce any Oxygen. Maybe I should add a specific demand reaction for it?
#Based on FVA, the model now produces some oxygen via the demand reaction. 

In [ ]:
#Model also facilitates transfer of metabolites unlike before.

In [ ]:
#Questions:
#Should I restrict Nutrient flow to the Bundle Sheath Cell only or should I allow flux to both the M and BS cell?
#Apparently kasi dito only the Bundle sheath cells produce any biomass
#This is to ensure that only CO2 and Light flux are the only limiting constraints to the system.